In [6]:
import hashlib
import time

class Block:
    def __init__(self, index, data, timestamp, previous_hash, difficulty, token=0):
        self.index = index
        self.data = data
        self.timestamp = timestamp
        self.previous_hash = previous_hash
        self.difficulty = difficulty
        self.token = token
        self.hash = self.calculate_hash()

    def calculate_hash(self):
        return hashlib.sha256(
            f"{self.index}{self.data}{self.timestamp}{self.previous_hash}{self.difficulty}{self.token}".encode()).hexdigest()

    def mine_block(self):
        print(f"Starting mining new block at index {self.index}...")
        start_time = time.time()
        while self.hash[:self.difficulty] != '0' * self.difficulty:
            self.token += 1
            self.hash = self.calculate_hash()

        end_time = time.time()
        print(f"New block mined: {self.hash}")
        print(f"Mining completed in {end_time - start_time} seconds.")

    def to_dict(self):
        return {
            'index': self.index,
            'timestamp': self.timestamp,
            'data': self.data,
            'previous_hash': self.previous_hash,
            'hash': self.hash,
            'difficulty': self.difficulty,
            'token': self.token
        }    

class Blockchain:
    def __init__(self, creation_interval, adjustment_interval):
        self.difficulty = 1
        self.chain = [self.create_genesis_block()]
        self.block_creation_interval = creation_interval
        self.adjustment_interval = adjustment_interval

    def create_genesis_block(self):
        genesis_block = Block(0, 'Genesis Block', time.time(), '0', self.difficulty)
        genesis_block.mine_block()
        return genesis_block

    def get_last_block(self):
        return self.chain[-1]

    def add_block(self, new_block):
        if self.is_valid_new_block(new_block, self.get_last_block()):
            self.chain.append(new_block)
            self.difficulty = self.adjust_difficulty()

    def is_valid_new_block(self, new_block, previous_block):
        if previous_block.index + 1 != new_block.index:
            return False
        if new_block.calculate_hash() != new_block.hash:
            return False
        if not self.validate_timestamp(new_block):
            return False
        return True

    def adjust_difficulty(self):
        if len(self.chain) % self.adjustment_interval == 0:
            return self.calculate_adjusted_difficulty()
        return self.difficulty

    def calculate_adjusted_difficulty(self):
        last_adjustment_block = self.chain[-self.adjustment_interval]
        expected_time = self.block_creation_interval * self.adjustment_interval
        actual_time = self.get_last_block().timestamp - last_adjustment_block.timestamp
        if actual_time < expected_time / 2:
            return last_adjustment_block.difficulty + 1
        elif actual_time > expected_time * 2:
            return max(1, last_adjustment_block.difficulty - 1)
        else:
            return last_adjustment_block.difficulty
            

    def calculate_cumulative_difficulty(self):
        return sum([2**block.difficulty for block in self.chain])

    def validate_timestamp(self, new_block):
        current_time = time.time()
        if new_block.timestamp > current_time + 60:
            return False
        if len(self.chain) > 1 and new_block.timestamp < self.get_last_block().timestamp - 60:
            return False
        return True
    

    

In [7]:
import socket
import threading
import json

class MessageType:
    NEW_BLOCK = 'new_block'
    REQUEST_CHAIN = 'request_chain'
    RESPONSE_CHAIN = 'response_chain'
    UPDATE_DIFFICULTY = 'update_difficulty'

class P2PNode:
    def __init__(self, host, port):
        self.host = host
        self.active = True
        self.mining_active = False
        self.port = port
        self.blockchain = Blockchain(10, 10)  
        self.peers = []
        self.server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.server.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        self.server.bind((self.host, self.port))
        self.server.listen(10)

    def start_server(self):
        threading.Thread(target=self.listen_for_incoming_connections, daemon=True).start()

    def listen_for_incoming_connections(self):
        while self.active:  
            try:
                client, address = self.server.accept()
                threading.Thread(target=self.handle_client, args=(client,)).start()
            except socket.timeout:  
                continue
            except Exception as e:
                print(f"Error accepting new connection: {e}")


    def mine_single_block(self, data):
        print("Mining a new block...")
        last_block = self.blockchain.get_last_block()
        new_index = last_block.index + 1
        new_timestamp = time.time()
        new_block = Block(
            index=new_index,
            data=data,
            timestamp=new_timestamp,
            previous_hash=last_block.hash,
            difficulty=self.blockchain.difficulty
        )
        new_block.mine_block()
        self.blockchain.add_block(new_block)
        print(f"Block {new_index} has been mined and added to the blockchain.")
        
        block_data = new_block.to_dict()
        self.broadcast(json.dumps({'type': MessageType.NEW_BLOCK, 'data': block_data}))
        self.broadcast_difficulty_update()            

    def handle_client(self, client):
        while True:
            try:
                message = client.recv(1024).decode()
                if message:
                    self.process_message(client, message)
            except Exception as e:
                print(f"Error handling client: {e}")
                client.close()
                break

    def process_message(self, client, message):
        message_data = json.loads(message)
        message_type = message_data['type']

        if message_type == MessageType.NEW_BLOCK:
            new_block_data = message_data['data']
            new_block = Block(
                new_block_data['index'],
                new_block_data['data'],
                new_block_data['timestamp'],
                new_block_data['previous_hash'],
                new_block_data['difficulty'],
                new_block_data['token']
            )

            if new_block.difficulty == self.blockchain.difficulty:
                if self.blockchain.is_valid_new_block(new_block, self.blockchain.get_last_block()):
                    self.blockchain.add_block(new_block)
                    print(f"New block added to the blockchain: {new_block.index}")
                else:
                    print("Received invalid block.")
            else:
                print("Received block with mismatched difficulty. Requesting current chain.")
                self.request_chain_from_peers()

        elif message_type == MessageType.REQUEST_CHAIN:
            chain_data = [block.to_dict() for block in self.blockchain.chain]
            response = json.dumps({'type': MessageType.RESPONSE_CHAIN, 'data': chain_data})
            client.send(response.encode())

        elif message_type == MessageType.RESPONSE_CHAIN:
            received_chain_data = message_data['data']
            received_chain = []
            for block_data in received_chain_data:
                new_block = Block(
                    index=block_data['index'],
                    data=block_data['data'],
                    timestamp=block_data['timestamp'],
                    previous_hash=block_data['previous_hash'],
                    difficulty=block_data['difficulty'],
                    token=block_data['token']
                )
                new_block.hash = block_data['hash']
                received_chain.append(new_block)

            if self.validate_and_replace_chain(received_chain):
                print("Blockchain updated from peer")
            else:
                print("Received blockchain is invalid")

        elif message_type == MessageType.UPDATE_DIFFICULTY:
            self.blockchain.difficulty = message_data['difficulty']
            print(f"Difficulty updated to {self.blockchain.difficulty}")        

    def broadcast_difficulty_update(self):
        message = json.dumps({'type': MessageType.UPDATE_DIFFICULTY, 'difficulty': self.blockchain.difficulty})
        self.broadcast(message)

    def is_connection_alive(self, client):
        try:
            client.send(b'')
            return True
        except socket.error:
            return False

    def display_connected_peers(self):
        connected_peers = []
        for peer in self.peers[:]:  
            if self.is_connection_alive(peer):
                peer_address = peer.getpeername() 
                connected_peers.append(f"{peer_address[0]}:{peer_address[1]}")
            else:
                self.peers.remove(peer)
        return connected_peers
  

    def mine_new_block(self, data):
        if not self.mining_active:
            return
        last_block = self.blockchain.get_last_block()
        new_index = last_block.index + 1
        new_timestamp = time.time()
        new_block = Block(
            index=new_index,
            data=data,
            timestamp=new_timestamp,
            previous_hash=last_block.hash,
            difficulty=self.blockchain.difficulty
        )
        new_block.mine_block()
        self.blockchain.add_block(new_block)
        print(f"Block {new_index} has been mined and added to the blockchain.")

        block_data = new_block.__dict__
        self.broadcast(json.dumps({'type': MessageType.NEW_BLOCK, 'data': block_data}))
        self.broadcast_difficulty_update()


    def validate_and_replace_chain(self, new_chain):
        if len(new_chain) <= len(self.blockchain.chain):
            print("Received blockchain is not longer than the current blockchain.")
            return False

        if self.is_valid_chain(new_chain) and all(block.difficulty == self.blockchain.difficulty for block in new_chain):
            self.blockchain.chain = new_chain
            self.blockchain.difficulty = new_chain[-1].difficulty
            self.broadcast_difficulty_update()
            print("Blockchain replaced with a better chain.")
            return True
        else:
            print("Received blockchain is invalid or has mismatched difficulties.")
            return False
    
    def display_blockchain(self):
        print("Current Blockchain:")
        for block in self.blockchain.chain:
            print(f"Index: {block.index}")
            print(f"Timestamp: {block.timestamp}")
            print(f"Data: {block.data}")
            print(f"Previous Hash: {block.previous_hash}")
            print(f"Hash: {block.hash}")
            print(f"Difficulty: {block.difficulty}")
            print(f"Token: {block.token}\n")

    def get_blockchain(self):
        return [block.to_dict() for block in self.blockchain.chain]
   
    def request_chain_from_peers(self):
        message = json.dumps({'type': MessageType.REQUEST_CHAIN})
        self.broadcast(message)
    
    def is_valid_chain(self, chain):
        if chain[0].calculate_hash() != self.blockchain.chain[0].calculate_hash():
            return False

        for i in range(1, len(chain)):
            if not self.blockchain.is_valid_new_block(chain[i], chain[i - 1]):
                return False

        return True
    
    def start_mining(self, data):
        self.mining_active = True
        threading.Thread(target=self.mine_continuously, args=(data,), daemon=True).start()

    def stop_mining(self):
        self.mining_active = False
    
    def mine_continuously(self, data):
        while self.mining_active:
            self.mine_new_block(data)
            time.sleep(1)

    def stop_server(self):
        print("Stopping server...")
        self.active = False  

        for client in self.peers:
            try:
                client.close()
            except Exception as e:
                print(f"Error closing client connection: {e}")

        try:
            self.server.close()
        except Exception as e:
            print(f"Error closing server socket: {e}")

        print("Server stopped.")

    def connect_to_node(self, host, port):
        client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        try:
            client.connect((host, port))
            self.peers.append(client)
            threading.Thread(target=self.handle_client, args=(client,)).start()
            self.request_chain_from_peers()
            print(f"Connected to node: {host}:{port}")
        except Exception as e:
            print(f"Error connecting to node: {host}:{port}, {e}")

    def is_peer_alive(self, peer):
        try:
            peer.send(b'')
            return True
        except socket.error:
            return False

    def broadcast(self, message):
        for peer in self.peers[:]:  
            if not self.is_peer_alive(peer):
                self.peers.remove(peer)
                continue
            try:
                peer.send(message.encode())
            except socket.error as e:
                print(f"Error broadcasting to peer: {e}")
                self.peers.remove(peer)



#node = P2PNode('localhost', 8000)
#node.start_server()

# For the second node
#second_node = P2PNode('localhost', 8001)
#second_node.start_server()

# Connect the second node to the first
#second_node.connect_to_node('localhost', 8000)



In [14]:
node = P2PNode('localhost', 8000)
node.start_server()

# For the second node
second_node = P2PNode('localhost', 8001)
second_node.start_server()

# Connect the second node to the first
second_node.connect_to_node('localhost', 8000)

In [18]:
node.stop_server()

Stopping server...
Error accepting new connection: [Errno 53] Software caused connection abort
Server stopped.


Error handling client: [Errno 54] Connection reset by peer


In [9]:
from flask import Flask, render_template, request, jsonify, redirect, url_for
import threading

app = Flask(__name__)
nodes = {}  
@app.route('/')
def home():
    return render_template('index.html')

@app.route('/create_node', methods=['POST'])
def create_node():
    port = int(request.json.get('port'))
    if port in nodes:
        return jsonify({'message': 'Node already exists on this port', 'success': False})
    
    try:
        node = P2PNode('164.8.160.177', port)
        threading.Thread(target=node.start_server, daemon=True).start()
        nodes[port] = node
        return jsonify({'message': f'Node created on port {port}', 'success': True})
    except Exception as e:
        return jsonify({'message': str(e), 'success': False})

@app.route('/node/<int:port>')
def node_page(port):
    if port not in nodes:
        return redirect(url_for('home'))
    return render_template('node.html', port=port)

@app.route('/display_blockchain', methods=['POST'])
def display_blockchain():
    port = int(request.json.get('port'))
    if port in nodes:
        blockchain_info = nodes[port].get_blockchain()
        print(blockchain_info)
        return jsonify({'blockchain': blockchain_info, 'success': True})
    return jsonify({'message': 'Node not found', 'success': False})

@app.route('/stop_node', methods=['POST'])
def stop_node():
    port = int(request.json.get('port'))
    if port in nodes:
        try:
            nodes[port].stop_server()  
            del nodes[port]  
            return jsonify({'message': 'Node stopped successfully', 'success': True})
        except Exception as e:
            return jsonify({'message': str(e), 'success': False})
    else:
        return jsonify({'message': 'Node not found', 'success': False})


@app.route('/connect_node', methods=['POST'])
def connect_node():
    data = request.json
    current_node_port_str = data.get('host') 
    target_port_str = data.get('port') 

    if current_node_port_str is None or target_port_str is None:
        return jsonify({'message': 'Current node port or target port not provided', 'success': False})

    if not all([current_node_port_str.isdigit(), target_port_str.isdigit()]):
        return jsonify({'message': 'Ports must be numbers', 'success': False})

    current_node_port = int(current_node_port_str)
    target_port = int(target_port_str)

    if current_node_port not in nodes:
        return jsonify({'message': 'Current node not found', 'success': False})

    try:
        nodes[current_node_port].connect_to_node('164.8.160.177', target_port)
        return jsonify({'message': 'Connected to node successfully', 'success': True})
    except Exception as e:
        return jsonify({'message': f'Error connecting to node: {e}', 'success': False})


@app.route('/get_connected_peers', methods=['GET'])
def get_connected_peers():
    port = int(request.args.get('port'))
    if port in nodes:
        try:
            peers = nodes[port].display_connected_peers()
            return jsonify({'peers': peers, 'success': True})
        except Exception as e:
            return jsonify({'message': str(e), 'success': False})
    else:
        return jsonify({'message': 'Node not found', 'success': False})


@app.route('/mine_block', methods=['POST'])
def mine_block():
    data = request.json
    port = int(data.get('port'))
    block_data = data.get('data')

    if port in nodes:
        try:
            nodes[port].mine_single_block(block_data)
            return jsonify({'message': 'New block mined successfully', 'success': True})
        except Exception as e:
            return jsonify({'message': str(e), 'success': False})
    else:
        return jsonify({'message': 'Node not found', 'success': False})


@app.route('/start_mining', methods=['POST'])
def start_mining():
    data = request.json
    port = int(data.get('port'))
    block_data = data.get('data')
    if port in nodes:
        nodes[port].start_mining(block_data)
        return jsonify({'message': 'Mining started', 'success': True})
    return jsonify({'message': 'Node not found', 'success': False})

@app.route('/stop_mining', methods=['POST'])
def stop_mining():
    data = request.json
    port = int(data.get('port'))
    if port in nodes:
        nodes[port].stop_mining()
        return jsonify({'message': 'Mining stopped', 'success': True})
    return jsonify({'message': 'Node not found', 'success': False})


if __name__ == '__main__':
    app.run(host='164.8.160.177', port=5002)



 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://164.8.160.177:5002
Press CTRL+C to quit
164.8.160.177 - - [22/Jan/2024 16:19:26] "GET / HTTP/1.1" 200 -
164.8.160.177 - - [22/Jan/2024 16:19:26] "GET /favicon.ico HTTP/1.1" 404 -
164.8.160.177 - - [22/Jan/2024 16:19:30] "POST /create_node HTTP/1.1" 200 -


Starting mining new block at index 0...
New block mined: 068f55a6b6bfeeff4a46bfb42f33086dda6c468d92f3b472e7f7fdf748795535
Mining completed in 0.00011801719665527344 seconds.


164.8.160.177 - - [22/Jan/2024 16:19:32] "GET /node/8000 HTTP/1.1" 200 -
164.8.160.177 - - [22/Jan/2024 16:28:58] "POST /display_blockchain HTTP/1.1" 200 -


[{'index': 0, 'timestamp': 1705936770.613377, 'data': 'Genesis Block', 'previous_hash': '0', 'hash': '068f55a6b6bfeeff4a46bfb42f33086dda6c468d92f3b472e7f7fdf748795535', 'difficulty': 1, 'token': 18}]


...

164.8.160.177 - - [22/Jan/2024 17:07:01] "POST /display_blockchain HTTP/1.1" 200 -


[{'index': 0, 'timestamp': 1705937523.810218, 'data': 'Genesis Block', 'previous_hash': '0', 'hash': '0098f0158c6115ca28be6bbdfaf4dfb1b66df4f883c8eebb43cca30900687563', 'difficulty': 1, 'token': 28}, {'index': 1, 'timestamp': 1705939049.978529, 'data': 'Number of people detected: 5, Location: Latitude: 46.55978, Longitude: 15.6382109', 'previous_hash': '0098f0158c6115ca28be6bbdfaf4dfb1b66df4f883c8eebb43cca30900687563', 'hash': '0ae3518d3fea8f516e7beec29f53c537363fe83459dfd6fee1cce2c77ded1de3', 'difficulty': 1, 'token': 0}]


164.8.160.177 - - [22/Jan/2024 17:09:39] "POST /stop_node HTTP/1.1" 200 -


Stopping server...
Error accepting new connection: [Errno 53] Software caused connection abort
Server stopped.


164.8.160.177 - - [22/Jan/2024 17:09:41] "GET / HTTP/1.1" 200 -
